## Compare Two Different Datasets using Wilcoxon Rank Sum Test

This noteboook allows you to compare two sets of textual data to one another to discover how, and how much, they differ. The notebook performs this comparison using the Wilcoxon rank sum test, a statistical test that determines if two samples (i.e., the relative frequencies of a specific word in two different datasets) are taken from populations that are significantly different from one another (meaning, if they have different distributions). When used to compare word frequency data from two different datasets, it helps you to determine what the most "significant" words in each dataset are.

See this module's `README.md` file for more information about the data used and produced in this notebook.

## Settings

This code block imports required Python modules and the `compare_word_frequencies.py` script and sets some required variables. You will set other variables you need as you move through the notebook.

In [ ]:
# Python imports
import os
import csv
from pathlib import Path
from IPython.display import display, HTML

# Import scripts
%run scripts/compare_word_frequencies.py

# Define paths 
# directory this notebook is in
current_dir       = %pwd
# directory of this repo on your machine
module_dir        = str(Path(current_dir).parent)
# directory of repo data
data_dir          = module_dir + '/data'

## How to Use this Notebook

The `input` folder in this module contains all of the derivative files you need to reproduce the Wilcoxon rank sum tests we describe in the article. If you wish to use these files, skip directly to section 3.A of this notebook. If you wish to reproduce the steps we took to create these files, or to conduct your own experiments with our data, begin with section 1.

## 1. Select Data

In this section, you can use provided lists of filenames to produce doc-terms files of a selection of documents from a larger collection. Using the provided lists of filenames will reproduce our experiments. Alternatively, you can use our doc-terms files to randomly select other groups of documents. You should run the code under EITHER section 1.A OR section 1.B to proceed.

NOTE: the code in this notebook is not optimized for large collections of data. As a result, we strongly recommend that you compare **2 randomly selected smaller samples** of a collection's data. Your two samples should be roughly equally sized. Anything under ~4000 documents should work.

### 1.A. Reproduce our experiment
Use lists of filenames to select the same documents that we did from a larger collection. The code below will grab the documents on each list from the collection's doc-terms file. It will create 2 new doc-terms files in the `input` directory. The order of the documents in these files will not match the order of documents in the provided files that we used (`c33-hum-docterms-500sample.txt` and `c33-sci-docterms-500sample.txt`), but they will contain the exact same documents (the order does not matter). The lists of files and the names of the new files the code will create are defined in the cell below.

Setting the `comparison` variable below will set the rest of the required filenames for this notebook. 

The variables below are organized by comparison. The humanities-science comparison is selected by default. If you want to run the not humanities-not science or humanities-not humanities comparisons, comment out the hum-sci section and uncomment the appropriate comparison's section.

Variables:
* `doc-terms`: the doc-terms file of the larger collection you want to select data from. Set to the humanities-science comparison by default.
* `filenames_c1`: the filenames of the files that we tested for category 1 (for example, articles classified as being about the humanities).
* `filenames_c2`: the filenames of the files that we tested for category 2 (for example, articles classified as being about science).
* `docterms_c1`: the doc-terms file for category 1 that the code below will create.
* `docterms_c2`: the doc-terms file for category 2 that the code below will create.

In [ ]:
# options are 'hum-sci', 'not-hum-not-sci', 'hum-not-hum'
comparison = 'hum-not-hum'
reproduce = True

collection, filenames_c1, filenames_c2, docterms_c1, docterms_c2 = set_comparison(comparison, reproduce, data_dir)


Now use these lists to grab the specific documents you want from the corpus you selected. After you run the below cell, skip to section 2.

In [ ]:
get_bags(filenames_c1, filenames_c2, collection, docterms_c1, docterms_c2)


### 1.B. Randomly select other documents from our data to test 
If, instead of reproducing our experiment, you would like to randomly select x number of documents from each category (for example, articles about the humanities and articles about science) in a larger collection, run the cells below. The code below uses the lists of filenames for articles classified as being about the humanities and articles classified as being about science located in the `data/tables` folder in this repo. It grabs a random number of documents from each of these categories (whatever you define as the value for the `selection` variable below) and produces 2 new doc-terms files containing just these randomly selected documents, one for each category. It will create 2 new doc-terms files in the `input` directory.

First, set the `selection` variable to the number of documents you want to select:

`selection = 500`

In [ ]:
selection = 500

Next, configure the filenames of the new doc-terms files you want to create.

In [ ]:
# hum-sci, not-hum-not-sci, hum-not-hum
comparison = 'hum-not-hum'
reproduce = False

collection, filenames_c1, filenames_c2, docterms_c1, docterms_c2 = set_comparison(comparison, reproduce, data_dir)


Now use these lists to grab a random sample of from the corpus you selected. After you run the below cell, go to section 2.

In [ ]:
get_random_sample(selection, filenames_c1, filenames_c2, collection, docterms_c1, docterms_c2)


## 2. Prepare Data for Test

**Now you are ready to calculate word frequency data for documents in each category.**

First, run the `findFreq` function to find the raw and the relative frequencies for each word in your 2 datasets.

In [ ]:
df1_relative, df1_freqs = findFreq(docterms_c1)
print("Completed first dataset.")

df2_relative, df2_freqs = findFreq(docterms_c2)
print("Completed second dataset.")


**Only include words that occur x number of times in each dataset.** In order to eliminate words that may seem "distinctive" to each dataset only because they occur very infrequently, the following cells use Python's `pandas` module to manipulate the raw and relative frequency dataframes returned by the above functions so that they only include words that occur x number of times in each dataset. First, you perform some dataframe manipulations to convert every `NaN` value to `0` across each dataset, add a `total_count` column to the raw frequency dataframes that lists the total number of times each word appears in that dataframe, and sort the raw frequency dataframes by highest to lowest total count values to find the top words in each dataset. Finally, to help set your threshold for inclusion, the function below determines the average total word count for each dataset. This value represents the average number of times any word appears in each dataset. 

In [ ]:
df1_relative, df1_freqs, df2_relative, df2_freqs = edit_freq_dataframes(df1_relative, df1_freqs, df2_relative, df2_freqs)


Based on what your average total word counts are as reported in the cell above, set your threshold for inclusion. The `threshold` variable represents the number of total times any word must appear in either dataset in order to be included. For instance, if you set `threshold = 5`, this means that any words that do not occur at least 5 times in either dataset will not be included in your analysis, while any words that do occur at least 5 times (or more), will be included in your analysis. 

The default threshold value is 5. If you wish to change the threshold number, here's how it should look:

`threshold = 6`

If you wish to NOT set a threshold for inclusion, set `threshold` to `False`.

`threshold = False`

In [ ]:
threshold = 5

Now use this threshold value to create new dataframes including only those words that occur at least x number of times (where x = `threshold`). The output of this cell will tell you how many words are now in each of your datasets (i.e., how many words meet the threshold you set). If you didn't set a threshold value above you still need to run the below cell to get variable names set right.

The code below will also save your newly created dataframes to disk so that you can reload them if you want to run this test again. The new files will be stored in the appropriate `input` subdirectory.

In [ ]:
c1_relative_csv, c2_relative_csv, c1_raw_csv, c2_raw_csv, vocablist = set_df_filenames(comparison, threshold)


Now run the code.

In [ ]:
df1, df2, words_c1, words_c2 = match_dataframes_and_save(threshold, df1_freqs, df1_relative, df2_freqs, df2_relative, c1_relative_csv, c2_relative_csv,
                                                                                    c1_raw_csv, c2_raw_csv)


Finally, run the code below to create the vocabulary list for both of your datasets. This is a list of all of the words that appear at least x times in either of your datasets (where x = threshold). The function below will produce a plain text file that will be saved in the appropriate `input` subdirectory in this module. 

**After running the below cell, skip to subsection 3.B below.**

In [ ]:
get_vocablist(df1, df2, words_c1, words_c2, vocablist)

## 3. Run Wilcoxon Rank Sum Test

You should run the code under EITHER subsection 3.A OR subsection 3.B below. After running the code under ONE of these subsections, continue on to subsection 3.C.

### 3.A. Reproduce our experiment 
If you want to use the provided derivative files to reproduce our experiment, run the cell below to set the variables you need. The values have been pre-set to the derivative files included in this repo (see the module's `README.md` file for more information about those files. You also need to decide what you want to call the results file, which will be saved to a csv file in the `results` folder in this module.

After running the below cells, skip to section 3.C.

In [ ]:
c1_relative_csv, c2_relative_csv, c1_raw_csv, c2_raw_csv, vocablist = set_df_filenames_existing(comparison)


In [ ]:
# set path to results csv
results_csv = 'results/wilcoxon-results.csv'

### 3.B. Continue with your own experiment 
If you have run through sections 1 and 2 of this notebook, you are now ready to conduct the Wilcoxon rank sum test. First, you need to decide what you want to call the results file, which will be saved to a csv file in the `results` folder in this module. 

After running the below cell, continue to section 3.C.

In [ ]:
results_csv = 'results/wilcoxon-results.csv'

### 3.C. Conduct the Wilcoxon rank sum test 
The code below will perform the test. The results will be saved in a csv file in the `results` folder in this module. See this modules `README.md` for more information about this results file.

In [ ]:
# run wilcoxon rank sum

wrs_test(c1_relative_csv, c1_raw_csv, c2_relative_csv, c2_raw_csv, vocablist, results_csv)
